# ePSproc/PEMtk dev - functional form dev
26/08/21

See prior notebook for basics: http://100.86.127.24/jupyter/user/paul/doc/tree/github/ePSproc/notebooks/in_progress/ePSdev-PEMtk_correlations_den-mats_basic-tests_220821.ipynb

## Setup

In [1]:
import sys
import os
from pathlib import Path
import numpy as np
# import epsproc as ep
import xarray as xr

import matplotlib.pyplot as plt

from datetime import datetime as dt
timeString = dt.now()

import epsproc as ep

# Multijob class dev code
from epsproc.classes.multiJob import ePSmultiJob

* sparse not found, sparse matrix forms not available. 
* natsort not found, some sorting functions not available. 
* pyevtk not found, VTK export not available. 


In [2]:
epDemoDataPath = Path(ep.__path__[0]).parent/'data'/'photoionization'

/home/paul/anaconda3/envs/epsdev-xr15/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Load test data

In [3]:
# Class dev code
from epsproc.classes.multiJob import ePSmultiJob
from epsproc.classes.base import ePSbase

# Instantiate class object.
# Minimally this needs just the dataPath, if verbose = 1 is set then some useful output will also be printed.
data = ePSbase(epDemoDataPath, verbose = 1)

# ScanFiles() - this will look for data files on the path provided, and read from them.
data.scanFiles()

/home/paul/github/ePSproc/epsproc/IO.py:657: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  attribs.append(['E', np.float(parseLineDigits(dumpSeg[3][2])[0]), 'eV'])
/home/paul/github/ePSproc/epsproc/IO.py:662: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/


*** Job orb8 details
Key: orb8
Dir /home/paul/github/ePSproc/data/photoionization, 1 file(s).
{   'batch': 'ePS ethylene, batch C2H4_1.0-100.0eV, orbital orb8_B3u',
    'event': 'HOMO ioinzation (B3u), wavefn run, 0.5:1:10.5, sph grid',
    'orbE': -10.32127880310325,
    'orbLabel': 'B3u'}

*** Job orb5 details
Key: orb5
Dir /home/paul/github/ePSproc/data/photoionization, 1 file(s).
{   'batch': 'ePS n2, batch n2_3sg_0.1-50.1eV, orbital A2',
    'event': ' N2 X-state (3sg-1)',
    'orbE': -17.34181645456815,
    'orbLabel': '3sg-1'}

*** Job orb11 details
Key: orb11
Dir /home/paul/github/ePSproc/data/photoionization, 1 file(s).
{   'batch': 'NO2 input file for ePolyScat',
    'event': ' Ionization of orbital 10 (A2)',
    'orbE': -14.378496492380059,
    'orbLabel': 'A2'}


## Function dev

Functions to calculate & plot density matrices...

In [8]:
#*** Dim functionality (see also lmPlot() and multiDimXrToPD() functions)

# Set imports
from epsproc.util import matEleSelector
from epsproc.util.misc import checkDims
# matEdimList, BLMdimList, dataTypesList, multiDimXrToPD
# checkDims = ep.util.misc.checkDims

def densityCalc(da, denDims = 'LM', selDims = None, sumDims = None, stackDims = None, 
                thres = None):
    
    # Set data
    daDen = matEleSelector(da, thres = thres, inds = selDims)  # TODO: pass **kwargs here?
                                                # Pass dims = denDims?
    
    # Check dims
    dimsIn = checkDims(daDen, refDims = denDims)
    
    print(dimsIn)

In [9]:
k = 'orb11'
matE = data.data[k]['matE']
selDims = {'Type':'L'}
thres = 1e-1


densityCalc(matE, selDims = selDims, thres = thres)

{'dataDims': ('LM', 'Eke', 'Sym', 'mu', 'it'), 'dataDimsUS': ('Eke', 'mu', 'it', 'l', 'm', 'Cont', 'Targ', 'Total'), 'refDims': 'LM', 'shared': [], 'extra': ['m', 'Eke', 'Total', 'l', 'Targ', 'mu', 'it', 'Cont'], 'stacked': ['Sym', 'LM'], 'stackedMap': {'Sym': FrozenList(['Cont', 'Targ', 'Total']), 'LM': FrozenList(['l', 'm'])}, 'invalid': ['M', 'L']}


In [10]:
matE.dims

/home/paul/anaconda3/envs/epsdev-xr15/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


('LM', 'Eke', 'Sym', 'mu', 'it', 'Type')

In [30]:
def checkDims2(data, refDims = []):
    """
    Check dimensions for a data array (Xarray) vs. a reference list (or dict).

    Parameters
    ----------
    data : Xarray
        Data array to check.
        
    refDims : str, list, optional, default = []
        Dims to check vs. input data array.
    
    Returns 
    -------
    
    dictionary 
        Containing:
        
        - stacked and unstacked dims
        - stacked dim mappings
        - intersection and differences vs. refDims
        

    TODO: check and order dims by size? Otherwise set return is alphebetical

    26/08/21 Added additional tests for stacked dims vs. ref.
             Bit messy, but left as-is to avoid breaking old code.
             In future should amalgamate stacked & unstacked tests & tidy output.
                
    23/08/21 Added stacked dim mapping output.
    11/05/21 Added handling for stacked dims.

    """
    
    # Force to list to avoid breaking *unpacking later
    if not isinstance(refDims, list):
        refDims = [refDims]
    
    dims = data.dims # Set dim list - this excludes stacked dims
    dimsUS = data.unstack().dims  # Set unstaked (full) dim list

    stackedDims = list(set(dims) - set(dimsUS))

    stackedDimsMap = {k:data.indexes[k].names for k in stackedDims}  # Get stacked dim mapping from indexes (Xarray/Pandas)

    # Check ref vs. full dim list (unstacked dims)
    sharedDims = list(set(dimsUS)&{*refDims})  # Intersection
    extraDims = list(set(dimsUS) - {*refDims})  # Difference
    invalidDims = list({*refDims} - set(dimsUS))  # Missing
    
    # 26/08/21 - added additional tests for stacked dims vs. ref, but may want to amalgamate with above?
    # TODO: tidy output too - separate dicts for stacked and unstacked dims?
    # Check ref vs. full dim list (stacked dims)
    sharedDimsStacked = list(set(dims)&{*refDims})  # Intersection
    extraDimsStacked = list(set(dims) - {*refDims})  # Difference
    invalidDimsStacked = list({*refDims} - set(dims))  # Missing

    # Test also missing dims overally
    missingDims = list({*refDims} - set(dimsUS) - set(dims))  # Missing
    
    return {'dataDims':dims, 'dataDimsUS':dimsUS, 'refDims':refDims, 
            'shared':sharedDims, 'extra':extraDims, 'invalid':invalidDims,
            'stacked':stackedDims, 'stackedMap':stackedDimsMap, 
            'stackedShared':sharedDimsStacked, 'stackedExtra':extraDimsStacked, 'stackedInvalid':invalidDimsStacked,
            'missing':missingDims}

In [31]:
checkDims2(matE, refDims = 'LM')

{'dataDims': ('LM', 'Eke', 'Sym', 'mu', 'it', 'Type'),
 'dataDimsUS': ('Eke', 'mu', 'it', 'Type', 'l', 'm', 'Cont', 'Targ', 'Total'),
 'refDims': ['LM'],
 'shared': [],
 'extra': ['m', 'Eke', 'Total', 'l', 'Targ', 'Type', 'mu', 'it', 'Cont'],
 'invalid': ['LM'],
 'stacked': ['Sym', 'LM'],
 'stackedMap': {'Sym': FrozenList(['Cont', 'Targ', 'Total']),
  'LM': FrozenList(['l', 'm'])},
 'stackedShared': ['LM'],
 'stackedExtra': ['Sym', 'Eke', 'Type', 'mu', 'it'],
 'stackedInvalid': [],
 'missing': []}

In [17]:
list({*refDims} - set(dimsUS) - set(dims))

NameError: name 'refDims' is not defined

In [20]:
refDims = 'LM'
print(*refDims)

L M


In [33]:
dimTest = checkDims2(matE, refDims = 'LM')